# **Exposure**

 <span style="font-family: times, Optima; font-size:11pt; color:black;">
The exposure information available in Samoa and Tonga is the compilation that was finalized in 2010 under the Pacific Catastrophe Risk Financing and Insurance Initiative (PCRAFI) project. The exposure database of PCRAFI contains country-specific information on assets, population, and some major infrastructure.<br>
    
    The assets included in the analysis are: buildings, roads, special infrastructures, crops, and population. 

> * **Buildings:** <br>
Buildings are represented as singular points containing the attributes of Occupation, Construction Type, Replacement Cost, Area, and Number of Stories.

> * **Road and special infrastructures:**<br> 
The special infrastructures are also singular points, but a buffer has been applied to represent the real area in a more realistic approach.

> * **Crops:**<br>
The crops layer contains the major cash-crops, their location, typology and replacement cost. This crops file, is a subset of a more comprehensive land cover geodatabase where other land uses categories were indexes. For the analysis performed here we have rely on the land use dataset and only account for the crops. 

> * **Population:**<br>
The population layer comes from SPC and is updated at 2020, it is a 90m raster, representing the population at each cell.

<span>
      

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import os.path as op
import sys

# basics
import glob
import xarray as xr
import pickle

# raster tools
from rasterio.crs import CRS
import geopandas as gpd
import json
import rioxarray

# kepler 
from keplergl import KeplerGl

**Load Exposure Layers**

In [ ]:
p_data = op.join(os.getcwd(), '..', 'data', 'riskscape', 'assets')

In [ ]:
a_file = open(op.join(p_data, '..', 'config_files', 'config_exposure_upolu.json'))
config_exp_up = json.load(a_file)

In [ ]:
build_exp = gpd.read_file(glob.glob(op.join(p_data, 'upolu_bld_all_buffer.shp'))[0])
land_exp = gpd.read_file(glob.glob(op.join(p_data, 'upolu_vector_final.shp'))[0])
roads_exp = gpd.read_file(glob.glob(op.join(p_data, 'upolu_roads.shp'))[0])
spec_exp = gpd.read_file(glob.glob(op.join(p_data, 'upolu_specials.shp'))[0])
people_exp = xr.open_rasterio(op.join(p_data, 'Population2020_proj.tif'))

**Reproject raster**

In [ ]:
crs_wgs84 = CRS.from_string('EPSG:4326') # WGS84

# Reproject the data using the crs object
people_84 = people_exp.rio.reproject(crs_wgs84)

In [ ]:
build_exp = build_exp.to_crs(epsg=4326)
land_exp = land_exp.to_crs(epsg=4326)
roads_exp = roads_exp.to_crs(epsg=4326)
spec_exp = spec_exp.to_crs(epsg=4326)

In [ ]:
build_exp = build_exp[['Value', 'Occ', 'geometry']]
land_exp = land_exp[['CLASS_NAME', 'geometry']]
roads_exp = roads_exp[['CLASS', 'geometry']]
spec_exp = spec_exp[['TYPE', 'geometry']]

In [ ]:
ds_people = people_84.to_dataset(name='density').drop('band').squeeze().drop('spatial_ref')
df_people = ds_people.where(ds_people.density != ds_people.density._FillValue).to_dataframe().dropna().reset_index()
df_people = df_people.rename(columns={'x':'Lon', 'y':'Lat'})

In [ ]:
# Load Hazard - flooding depth
map_1 = KeplerGl(height=700, show_docs=False, config=config_exp_up)

# Load Exposure - Assets: Buildings, Land and Roads
map_1.add_data(data=build_exp, name='Buildings Exposure')
map_1.add_data(data=land_exp, name='Land Exposure')
map_1.add_data(data=roads_exp, name='Roads Exposure')
map_1.add_data(data=spec_exp, name='Specials Exposure')
map_1.add_data(data=df_people, name='People EXposure')
map_1